In [18]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sqlalchemy import create_engine
import json
import pickle
import string

In [5]:
with open('../secrets.json') as file:
    secrets = json.load(file)
    connection_string = secrets['connection_string']
db = create_engine(connection_string)
df = pd.read_sql('select * from news_article', con=db)

load the tag lookups

In [10]:
with open('models/doc2vec/tag_lookup.pickle', 'rb') as f:
    tag_lookup = pickle.load(f)

In [16]:
def clean_headline(headline):
    stop_words = set(stopwords.words('english'))
    return [word for word in nltk.word_tokenize(headline) if word not in stop_words and word not in string.punctuation]

In [21]:
loaded_model = Doc2Vec.load('models/doc2vec/headline_model')

new_headline = df['headline'][1]
new_clean_headline = clean_headline(new_headline)
similar = loaded_model.docvecs.most_similar(positive=[loaded_model.infer_vector(new_clean_headline)],topn=6)
similar.pop(0)

print(f'{new_headline}\n')

# show similar headlines
for i, s in enumerate(similar):
    print(f'{i}: {tag_lookup[s[0]]}')

A third of COVID survivors suffer neurological or mental disorders: study

0: Saudi Arabia textbooks revised to be more tolerant - study
1: Dark hair was common among Vikings, genetic study confirms
2: COVID has worsened inequality even as the rich thrive: Oxfam
3: Lung damage found in COVID dead may shed light on 'long COVID' - study
4: Trained dogs were able to sniff out Covid-19 infections with 94% accuracy: study
